In [25]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

In [26]:
#定义一些训练超参数
batch_size = 64
epochs = 100
lr = 0.001
transform = transforms.ToTensor()

#使用tensorboard进行可视化
writer = SummaryWriter('log')  

In [27]:
#定义LeNet5神经网络结构
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Sequential( nn.Conv2d(1, 6, 5, 1, 2), nn.ReLU(),nn.MaxPool2d(kernel_size=2, stride=2))
        self.conv2 = nn.Sequential(nn.Conv2d(6, 16, 5),nn.ReLU(),nn.MaxPool2d(2, 2))
        self.fc1 = nn.Sequential(nn.Linear(16 * 5 * 5, 120),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(120, 84),nn.ReLU())
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        return x

In [28]:
#运用torchvison库的自带函数进行MNIST数据集的下载，然后进一步划分训练集和测试集的dataloader
trainset = torchvision.datasets.MNIST(root='data/',train=True,download=True,transform=transform)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=batch_size,shuffle=True,)


testset = torchvision.datasets.MNIST(root='data/',train=False,download=True,transform=transform)
testloader = torch.utils.data.DataLoader(testset,batch_size=batch_size,shuffle=False,)

In [29]:
#定义device以及loss_function等训练需要需要的条件
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
net = LeNet().to(device)
criterion = nn.CrossEntropyLoss() 
#optimizer = optim.Adam(net.parameters(), lr=lr)
optimizer = optim.SGD(net.parameters(), lr=lr,momentum=0.9)

In [30]:
#训练函数
def train_loop():
    for epoch in range(epochs):
        
        #进入训练
        sum_loss = 0.0
        correct = 0
        for data in tqdm(trainloader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)       #与模型放入同一训练器
            outputs = net(inputs)
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).sum()
            loss = criterion(outputs, labels)
            sum_loss += loss
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            

        writer.add_scalar('train_acc',correct / (len(trainloader)*batch_size),epoch)
        writer.add_scalar('train_loss',sum_loss,epoch)
            
            
        #进入测试
        with torch.no_grad():
            correct = 0
            for data in testloader:
                images, labels = data
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                correct += (predicted == labels).sum()
                
            writer.add_scalar('test_acc',correct/(len(testloader)*batch_size),epoch)
            test_acc = float((correct/(len(testloader)*batch_size)).cpu())
            print('第{}个epoch在测试集上识别准确率为:'.format(epoch+1),test_acc)


In [31]:
if __name__ =='__main__':
    train_loop()

100%|██████████| 938/938 [00:07<00:00, 127.66it/s]


第1个epoch在测试集上识别准确率为: 0.6892914175987244


100%|██████████| 938/938 [00:07<00:00, 130.42it/s]


第2个epoch在测试集上识别准确率为: 0.9115247130393982


100%|██████████| 938/938 [00:07<00:00, 129.66it/s]


...

第98个epoch在测试集上识别准确率为: 0.9852707386016846


100%|██████████| 938/938 [00:07<00:00, 127.71it/s]


第99个epoch在测试集上识别准确率为: 0.9856687784194946


100%|██████████| 938/938 [00:07<00:00, 130.91it/s]


第100个epoch在测试集上识别准确率为: 0.9858678579330444
